##### ### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2025 Semester 1

## Assignment 1: Scam detection with Naive Bayes


**Student ID(s):**     1462539


## 0. Set-up

Let us load data from `sms_supervised_train.csv`

In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Load data
data = pd.read_csv('data/sms_supervised_train.csv')

Now that we loaded out data, let us delete rows where `textPreprocessed` is empty by calling `data.dropna`. These instances do not have any features we can learn from. We can then tokenise (split) the already preprossessd text for our next stage.

In [6]:
# Split text into tokens
data.dropna(subset=['textPreprocessed'], inplace=True)
data['tokens'] = data['textPreprocessed'].apply(lambda x: x.split())

## 1. Supervised model training


## 2. Supervised model evaluation

## 3. Extending the model with semi-supervised training

## 4. Supervised model evaluation